In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Topic 1. Exploratory Data analysis with Pandas 

### Demonstration of main Pandas methods
   [Pandas](https://pandas.pydata.org/) is a Python library that provides extensive means for data analysis. Data Scientist often work with data stored in table formats like `.csv`, `.tsv`, or `.xlsx`. Pandas makes it very convenient to load, process, and analyze such tabular data using SQL-like  queries. In conjuction with `Matplotlib` and `Seaborn`, `Pandas` provides a wide range of opportunities for visual analysis of tabular data.

The main data structures in `Pandas` are implemented with **Series** and **DataFrame** classes. The former is one-dimensional indexed array of some fixed data type. The latter is a two-dimensional data structure- a table - where each column contains data of the same type. You can see it as dictionary of `Series` instances. `DataFrame` are great for representing real data: rows correspond to instances (examples, observations, etc.), and columns corresponf to features of these instances.

In [ ]:
import numpy as np
import pandas as pd

We'll demonstrate the main methods in action by analyzing a dataset on the churn rate of telecom operator clients. Let's read the data(using `read_csv`), and take a look at the first 5 lines using the `head` methods:

In [ ]:
df = pd.read_csv('../input/mlcourse/telecom_churn.csv')
df.head()

By default, Pandas displays 20 columns and 60 rows, So, if your DataFrame is bigger, use the `set_option` function as shown in the example below.

```python pd.set_option('display.max_columns',100) pd.set_option('display.max_rows',100)```
Recall that each rows corresponds to one client, an **instance**, and columns are **features** of this instance.


Let's have a look at data dimensionality, feature names, and feature types.

In [ ]:
df.shape

From the output, we can see that the table contains 3333 rows and 20 columns.


Now let's try printing out column names using `columns`:

In [ ]:
df.columns

We can use the `info()` method to output some general information about the dataframe:

In [ ]:
df.info()

`bool`, `int64`, `float64` and `object` are the data types of out features. We can see that one feature is logical(`bool`), 3 features are of type `object`, and 16 features are numeric. With this same method, we can easily see, if there are any missing values. Here, there are none because each column contains 3333 observations, the same number of rows we saw before with `shape`.



We can **change the column type** with the `astype` method. Let's apply this method to the `Churn` feature to convert it into `int64`

In [ ]:
df['Churn'] = df['Churn'].astype('int64')

The `describe` method shows basic statistical characteristics of each numerical feature(`int64` and `float64` types): number of non-missing values, mean, standard deviation, range. median, and 0.25 and 0.75 quartiles.

In [ ]:
df.describe()

In order to see statistics on non-numerical features, one has to explicitly indicate data types of interest in the `include` parameter.

In [ ]:
df.describe(include=['object','bool'])

For categorical (type `object`) and boolean (type `bool`) features we can use the `value_counts` method. Let's have a look at the distribution of `Churn`:

In [ ]:
df['Churn'].value_counts()

2850 users of 3333 are loyal; their `Churn` value is `0`. To calculate fractions, pass `normalize=True` to the `value_counts` function.

In [ ]:
df['Churn'].value_counts(normalize=True) #percentages of 0 and 1

### Sorting 
 A DataFrame can be sorted by the value of one of the variables (i.e columns). For example, we can sort by Total day charge (use `ascending=False` to sort in descending order):

In [ ]:
df.sort_values(by='Total day charge', ascending=False).head()

We can also sort by multiple columns:

In [ ]:
df.sort_values(by=['Churn','Total day charge'],ascending=[True,False]).head()

### Indexing and retrieving data
A DataFrame can be indexed in a few different ways.
To get a single column, you can use a `DataFrame['Name']` construction. Let's use this to answer a question about that column alone: **what is the proportion of churned users in our dataframe?**

In [ ]:
df['Churn'].mean()

14.5% is actually quite bad for a company; such a churn rate can make the company go bankrupt.
**Boolean indexing** with one column is also very convenient. The syntax is `df[P(df['Name'])]`, result of such indexing is the DataFrane consisting only of rows that satisfy the `P` condiction on the `Name` column.

Let's use it to answer the question:


**What are average values of numerical features for churned users?**

In [ ]:
df[df['Churn'] == 1].mean()

**How much time (on average) do churned users spend on the phone during daytime?**

In [ ]:
df[df['Churn'] == 1]['Total day minutes'].mean()

**What is the maximum length of international calls among loyal users(Churn == 0) who do not have an international plan?**

In [ ]:
df[(df['Churn'] == 0) & (df['International plan'] == 'No')]['Total intl minutes'].max()

DataFrames can be indexed by column name (label) or row name (index) or by the serial number of a row. The `loc` method is used for **indexing by name**, while `iloc()` is used for **indexing by number**.

In the first case below, we say "give us the values of the rows with index from 0 to 5(inclusive) and columns labeled from State to Area code (inclusive)". In the second case, we say "give us the values of the fisrt five rows in the first three columns"(as in a typical Python slice: the maximal value is not included).

In [ ]:
df.loc[0:5, 'State':'Area code']

In [ ]:
df.iloc[0:5,0:3]

If we need the first or the last line of the data frame, we can use the `df[:1]` or `df[-1]`

In [ ]:
df[-1:]

### Applying Functions to Cells, Columns and Rows
**To apply functions to each column, use apply():**

In [ ]:
df.apply(np.max)

The `apply` method can also be used to apply a function to each row. To do this, specify `axis=1` Lambda functions are very convenient in such scenarios. For example, if we need to select all states starting with W, we can do it like this:

In [ ]:
df[df['State'].apply(lambda state: state[0] == 'W')].head()

The `map` method can be used to **replace values in a column** by possing a dictionary of the form `{old_value: new_value}` as its arguent:

In [ ]:
d = {'No': False, 'Yes' : True}
df['International plan'] = df['International plan'].map(d)
df.head()

The same things can be done with the `replace` method:

In [ ]:
df = df.replace({'Voice mail plan':d})
df.head()

### Grouping 
In general, grouping data in Pandas works as follows:
    `df.groupby(by=grouping_columns)[columns_to_show].functions()`
   1. First, the `groupby` method divides the `grouping_columns` by their values. They become a new index in the resulting dataframe.
   2. Then, columns of interest are selected(`columns_to_show`), If `columns_to_show` is not included, all non groupby clauses will be included.
   3. Finally, one or several functions are applied to the obtained groups per selected columns.
Here is an example where we group the data accoring to the values of the `Churn` variable and display statistics of three columns in each group:


In [ ]:
columns_to_show = ['Total day minutes','Total eve minutes','Total night minutes']

df.groupby(['Churn'])[columns_to_show].describe(percentiles=[])

### Summary tables
Suppose we want to see how the observations in our sample are distributed in the context of two variables- `Churn` and `International plan`. To do so, we can build a **contingency table** using the `crosstab` method:

In [ ]:
pd.crosstab(df['Churn'], df['International plan'])

In [ ]:
pd.crosstab(df['Churn'], df['Voice mail plan'], normalize=True)

We can see that most of the users are loyal and do not use additional services (International Plan/Voice mail).
This will resemble **Pivot tables** to those familiar with Excel. And, of course, pivot tables are implemented in Pandas: the `pivot_table` method takes the following parameters:
* `values` - a list of variables to calculate statistics for,
* `index` - a list of variables to group data by,
* `arrfunc` - what statistics we need to calculate for groups, ex. sum, mean, max, min or something else.
Let's take a look at the average number of day, evening, and night calls by area code:

In [ ]:
df.pivot_table(['Total day calls','Total eve calls','Total night calls'],['Area code'], aggfunc='mean')

### DataFrame Transformations
Like many other things in Pandas, adding columns to a DataFrame is doable in many ways.
 For example, if we want to calculate the total number of calls for all users, let's create the `total_calls` Series and paste it into the DataFrame:
 

In [ ]:
total_calls = df['Total day calls'] + df['Total eve calls'] + \
              df['Total night calls'] + df['Total intl calls']
df.insert(loc=len(df.columns), column='Total calls', value=total_calls)
# loc parameter is the number of columns after which to insert the series object 
# we set it to len(df.columns) to paste it at the very end of the dataframe
df.head()

It is possible to add a column more easily without creating an intermediate Series instance:

In [ ]:
df['Total charge'] = df['Total day charge'] + df['Total eve charge'] + \
                     df['Total night charge'] + df['Total intl charge']
df.head()

To delete columns or rows, use the `drop` method, passing the required indexed and the `axis` parameter(`1` if you delete columns, and nothing or `0` if you delete rows). The `inplace` argument teels whether to change the original DataFrame. With `inplace=False`, the `drop` columns, With `inplace=True`, it alters the DataFrame.

In [ ]:
# get rid of just created columns
df.drop(['Total charge','Total calls'],axis=1,inplace=True)
# and here's how you can delete rows
df.drop([1,2]).head()

### 2. First attempt at predicting telecom churn
Let's see how churn rate is related to the *International plan* feature. We'll do this using a `crosstab` contingency table and also through visual analysis with `Seaborn` (however, visual analysis will be covered more thoroughly in the next article).

In [ ]:
pd.crosstab(df['Churn'], df['International plan'],margins=True)

In [ ]:
# some imports tp set up plotting
import matplotlib.pyplot as plt
%matplotlib inline
# pip install seaborn
import seaborn as sns
# Graphics in retina format are more sharp and Legible
%config InlineBackend.figure_format = 'retina'

In [ ]:
sns.countplot(x='International plan',hue='Churn', data=df)

We see that, with *international Plan*, the churn rate is much higher, which is an interesting observation! Perhaps large and poorly controlled expenses with international calls are very conflict-prone and lead to dissatisfaction among the telecom operator's customers.

Next, let's look at another important feature - Customer service calls. Let's also make a summary table and a picture

In [ ]:
pd.crosstab(df['Churn'],df['Customer service calls'], margins=True)

In [ ]:
sns.countplot(x='Customer service calls',hue='Churn',data=df)

Although it's not so obvious from the summary table, it's easy to see from the above plot that the churn rate increases sharply from 4 customer service calls and above.

Now let's add a binary feature to our DataFrame -`Customer service calls >3`. And once again, let's see how it relates to churn.

In [ ]:
df['Many_service_calls'] = (df['Customer service calls'] > 3).astype('int')

pd.crosstab(df['Many_service_calls'], df['Churn'], margins=True)

In [ ]:
sns.countplot(x='Many_service_calls',hue='Churn',data=df)

Let's construct another contingency table that related churn with both International plan and freshly created Many_service_calls.

In [ ]:
pd.crosstab(df['Many_service_calls'] & df['International plan'], df['Churn'])

Therefore, predicting that a customer is not loyal (Churn=1) in the case when the number of calls to the service center is greater than 3 and the International Plan is added (and predicting Churn=0 otherwise), we might expect an accuracy of 85.8% (we are mistaken only 464 + 9 times).

# Topic - Visual data analysis in Python
## [Link](https://www.kaggle.com/taruntiwarihp/visual-data-analysis-in-python-20-mins)
